# MTH 652: Advanced Numerical Analysis

## Homework Assignment 2

### <span style="color:red;">Write your name here</span>

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and code.
  * The use of ChatGTP, Copilot, or other AI assistants is **not allowed**
* The code must run in Colab or JupyterHub without errors.
  * Code that does not run will not receive any credit.
  * I suggest double-checking that your code runs properly in a new session. Sometimes code can be broken but appear to work because of old state in the notebook.

### Google Colab Instructions

* After opening this assignment in Google Colab, click on **"Copy to Drive"**
* Rename the notebook to `student_name_mth_652_assignment_2.ipynb`
    * ⚠️ In the above, replace `student_name` with your name!
* Enter your name above (in the cell below "Homework Assignment")!
* When you are ready to submit your assignment, select "File -> Download -> Download .ipynb" from the Colab menu
* Upload the downloaded `.ipynb` file to Canvas

### Assignment Goals

* The purpose of this assignment is to:
    1. Understand the role of the infinitesimal rigid motions in linear elasticity.
    2. Study the locking phenomenon in linear elasticity using MFEM.

### Written Questions

#### 1. (3 points)

Characterize the nullspace of the symmetric gradient operator $\nabla^s$ in $\mathbb{R}^2$ and $\mathbb{R}^3$.

**Hint:** in $\mathbb{R}^d$, consider functions of the form $\boldsymbol x \mapsto \boldsymbol Z \boldsymbol x$, where $\boldsymbol Z \in \mathbb{K}$, which is the space of $d \times d$ anti-symmetric matrices.

Relate this nullspace to the concept of rigid body motions and explain the connection to rotation matrices.

#### 2. (3 points)

For $\Omega \subseteq \mathbb{R}^2$, show that the space $\boldsymbol{H}^1(\Omega)$ can be decomposed into the sum
$$
    \boldsymbol{H}^1(\Omega) = \widehat{\boldsymbol{H}^1}(\Omega) + \boldsymbol{RM}
$$
where
$$
   \widehat{\boldsymbol H}^k(\Omega) = \left\{ \boldsymbol v \in \boldsymbol H^k(\Omega) : \int_\Omega \boldsymbol v \, dx = 0 \ \text{and}\ \int_\Omega \operatorname{rot} \boldsymbol v \, dx = 0 \right\}
$$
and $\boldsymbol{RM}$ is the space of infinitesimal rigid-body motions.

Recall that $\operatorname{rot} \boldsymbol v = - \partial v_1 / \partial y + \partial v_2 / \partial x$.

Hint: start by defining $\boldsymbol w = \boldsymbol c + b (y,-x)^T$, with
$$
    b = \frac{-1}{2|\Omega|} \int_\Omega \operatorname{rot} \boldsymbol v \, dx
$$
and
$$
    c = \frac{1}{|\Omega|} \int_\Omega (\boldsymbol v - b(y, -x)^T) \, dx
$$

#### 3. (3 points)

In $\mathbb{R}^2$, show that if $\boldsymbol w \in \boldsymbol{RM}$ vanishes on $\overline{\Gamma_D}$, where $\Gamma_D \subseteq \partial\Omega$ is a nonempty open set, then $\boldsymbol w \equiv \boldsymbol 0$.

Hint: it suffices to consider two distinct points $\boldsymbol x_1, \boldsymbol x_2 \in \Gamma_D$.

#### 4. (3 points)

Let $\boldsymbol V_h = [W_h]^2$ denote the vector-valued piecewise-linear vector-valued finite element space on a triangulation of $\Omega \subseteq \mathbb{R}^2$.

Characterize the subspace consisting of $\boldsymbol v_h \in \boldsymbol V_h$ such that $\nabla \cdot \boldsymbol v_h = 0$.

What about the space $\boldsymbol V_h^0 = \{ \boldsymbol v_h \in \boldsymbol V_h : \boldsymbol v_h|_{\Gamma_D} = 0 \}$?

### Programming Questions

#### 5. (2 points)

Find the function $\boldsymbol f(x,y)$ (depending on Lamé parameters $\mu$ and $\lambda$) such that
$$
    \boldsymbol u(x,y) = \begin{pmatrix}
        -\cos(\pi x) \sin(\pi y) \\
        \sin(\pi x) \cos(\pi y)
    \end{pmatrix}
$$
is the solution to the equations of linear elasticity (with, for example, pure displacement boundary conditions given by $\boldsymbol g_D = \boldsymbol u|_{\partial\Omega}$).

Write a function `rhs(mu, lambda_)` that **returns a function** `f(x,y)` that evaluates $f(x,y)$.

In [76]:
import numpy as np

def rhs(mu, lambda_):
    def f(x, y):
        return np.array([0.0, 0.0])
    return f

#### 6.

Read and understand the following function, which solves the linear elasticity problem with displacement boundary conditions in MFEM, taking `f`, `mu`, `lambda_` and `nx` as parameters.

The functions assumes that the input function $\boldsymbol f$ is the one corresponding to the right-hand side from Problem 5.

The number of Gauss-Seidel-preconditioned conjugate gradient iterations required to solve the linear system and the $L^2$-norm error compared with $\boldsymbol u(x,y)$ are returned.

In [77]:
import mfem.ser as mfem

def solve_elasticity(f, mu, lambda_, nx):
    # Polynomial degree
    order = 1

    # Square mesh
    mesh = mfem.Mesh(nx, nx, "TRIANGLE")
    dim = mesh.Dimension()

    # H^1 finite elements, with order = 1
    fec = mfem.H1_FECollection(order, dim)

    # Create the finite element space. Note that dim for the third argument
    # creates the space V_h = [W_h]^d, where W_h is the scalar space in H^1.
    fes = mfem.FiniteElementSpace(mesh, fec, dim)

    # Get the list of essential DOFs
    ess_tdof_list = mfem.intArray()
    fes.GetBoundaryTrueDofs(ess_tdof_list)

    # Define the Lamé parameters (constants according to the input parameters)
    lambda_coeff = mfem.ConstantCoefficient(lambda_)
    mu_coeff = mfem.ConstantCoefficient(mu)

    # Create the elasticity bilinear form with given lambda and mu
    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ElasticityIntegrator(lambda_coeff, mu_coeff))
    a.Assemble()

    class ForceCoeff(mfem.VectorPyCoefficient):
        def EvalValue(self, x):
            return f(x[0], x[1])

    class ExactCoeff(mfem.VectorPyCoefficient):
        def EvalValue(self, x):
            return np.array([
                -np.cos(np.pi * x[0]) * np.sin(np.pi * x[1]),
                np.sin(np.pi * x[0]) * np.cos(np.pi * x[1])
            ])

    u_coeff = ExactCoeff(2)

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.VectorDomainLFIntegrator(ForceCoeff(2)))
    b.Assemble()

    x = mfem.GridFunction(fes)
    x.Assign(0.0)

    bdr_attr = mfem.intArray(4)
    bdr_attr.Assign(1)
    x.ProjectBdrCoefficient(u_coeff, bdr_attr)

    # Form the linear system: eliminate boundary conditions, etc.
    A = mfem.SparseMatrix()
    B = mfem.Vector()
    X = mfem.Vector()
    a.FormLinearSystem(ess_tdof_list, x, b, A, X, B)

    # Gauss-Seidel preconditioner
    M = mfem.GSSmoother(A)

    # Preconditioned conjugate-gradient solver
    cg = mfem.CGSolver()
    cg.SetRelTol(1e-8)
    cg.SetMaxIter(5000)
    cg.SetPrintLevel(0)
    cg.SetPreconditioner(M)
    cg.SetOperator(A)
    cg.Mult(B, X)

    a.RecoverFEMSolution(X, b, x)

    # Return number of CG iterations and L2 error compared with true solution
    return (cg.GetNumIterations(), x.ComputeL2Error(u_coeff))


#### 7. Locking (6 points)

Run the above function, varying the Lamé parameter $\lambda \in \{ 1, 100, 10^5 \}$ on meshes of size $n_x \in \{ 10, 20, 40, 80 \}$.

What can you say about the $L^2$ convergence rate for each of these cases?

What happens for large $\lambda$? Do you have a (perhaps partial) explanation for this? Is this consistent with the finite element error estimates for elliptic problems?

What happens to the number of CG iterations with respect to both $n_x$ and $\lambda$? Do you have an explanation for why this happens?